In [12]:
import requests
from newspaper import Article

# -------- CONFIG --------
SERPER_API_KEY = "b6377b6d2d07fe6a6f334f03986986351846d4bb"  # 🔁 Replace with your actual key
OLLAMA_API_URL = "http://localhost:11434/api/generate"

In [2]:
import re 

In [3]:
pip install playwright


^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
playwright install


SyntaxError: invalid syntax (2310654265.py, line 1)

In [ ]:
!pip install selenium


In [2]:
def ask_llama3_locally(prompt):
    try:
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        response = requests.post(
            OLLAMA_API_URL,
            json={"model": "llama3", "prompt": prompt, "stream": False},
            headers=headers,
            verify=False  # Disable SSL verification for ngrok URL
        )
        if response.status_code == 200:
                answer_text = response.json().get("response", "[No response found]")
                return answer_text  # ✅ Return only the clean text
        else:
            return f"[HTTP {response.status_code}] Error from Llama3"
    except Exception as e:
        return f"[❌ Error calling Llama3: {e}]"

In [3]:
def is_url_trusted_by_ai(url):
    prompt = f"""You are a trustworthiness evaluator.

Decide whether the following website is generally considered a **professional and trustworthy source of information** for serious research, financial news, or due diligence.

Do **not** include websites that:
- Are editable by the public (like Wikipedia, Reddit, Quora)
- Are known for unverified or user-generated content

Only reply with `yes` or `no`.

### Website:
{url}

### Is this source trusted?
"""

    try:
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        response = requests.post(
            OLLAMA_API_URL,
            json={"model": "llama3", "prompt": prompt, "stream": False},
            headers=headers,
            verify=False
        )
        if response.status_code == 200:
            verdict = response.json().get("response", "").strip().lower()
            print(f"🤖 AI verdict for {url}: {verdict}")
            return "yes" in verdict
        else:
            return False
    except Exception as e:
        print(f"[AI trust check failed for {url}]: {e}")
        return False


In [19]:
def web_search_serper(query, max_results):
    headers = {
        "X-API-KEY": SERPER_API_KEY
    }
    json_data = {
        "q": query
    }
    response = requests.post("https://google.serper.dev/search", headers=headers, json=json_data)

    if response.status_code != 200:
        print(f"❌ Serper API error: {response.status_code}")
        return []

    data = response.json()
    results = data.get("organic", [])
    urls = []
    for item in results:
        url = item["link"]
        print(f"🔍 Evaluating source: {url}")
        if is_url_trusted_by_ai(url):
            urls.append(url)
        if len(urls) >= max_results:
            break
    return urls


In [5]:
def classify_question_type(query):
    classification_prompt = f"""You are a classification assistant.

Decide if the following question requires a short, direct answer (like a number, name, or fact), or a detailed, multi-paragraph summary.

Respond with only one word: `short` or `detailed`.

### Question:
{query}

### Answer Style:"""

    # Send to LLaMA3
    try:
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
        response = requests.post(
            OLLAMA_API_URL,
            json={"model": "llama3", "prompt": classification_prompt, "stream": False},
            headers=headers,
            verify=False
        )
        if response.status_code == 200:
            style = response.json().get("response", "").strip().lower()
            return "short" if "short" in style else "detailed"
        else:
            return "detailed"
    except:
        return "detailed"


In [ ]:
def display_final_answer(answer, query, urls):
    print("\n" + "="*80)
    print(f"🧠 Answer to: {query}")
    print("="*80)
    print("tetete"+answer)

    print("\n🔗 Sources Used:")
    for url in urls:
        print(f" - {url}")
    print("="*80)


In [15]:
def classify_query_complexity(query):
    classification_prompt = f"""
You are a helpful assistant specializing in classifying question complexity.

Classify the following user question into one of three categories:
- "simple" (if the answer can easily be found in 1-2 sources, factual)
- "moderate" (if it requires multiple sources but not deep reasoning)
- "complex" (if it needs aggregation, comparison, reasoning, or analysis)

⚡ Important:
- ONLY respond with one word: simple, moderate, or complex.
- Do NOT add any extra explanation.

Question:
"{query}"
"""
    response = ask_llama3_locally(classification_prompt)
    print(f"🤖 AI complexity classification for '{query}': {response}")
    return response.strip().lower()

def estimate_number_of_urls(query):
    classification = classify_query_complexity(query)

    if classification == "simple":
        return 5
    elif classification == "moderate":
        return 7
    elif classification == "complex":
        return 10
    else:
        return 5  # fallback just in case

# Example usage


In [24]:
def extract_readable_text(url):
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from bs4 import BeautifulSoup
    import time

    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36")

    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(15)

    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined});"
    })

    try:
        driver.get(url)
        time.sleep(2)  # Let the page start rendering

        scroll_pause_time = 1.0
        last_height = driver.execute_script("return document.body.scrollHeight")

        for _ in range(2):  # Scroll less times
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        time.sleep(1)

        html = driver.page_source
    except Exception as e:
        print(f"Page load failed: {e}")
        html = driver.page_source
    finally:
        driver.quit()

    soup = BeautifulSoup(html, "html.parser")

    texts = soup.find_all(string=True)

    output = ''
    blacklist = ['[document]', 'noscript', 'header', 'html', 'meta', 'head', 'input', 'script', 'style', 'iframe', 'svg']

    for t in texts:
        if t.parent.name not in blacklist and t.strip():
            output += '{} '.format(t.strip())

    return output


def clean_text(text):
    # Pre-clean obvious junk
    junk_phrases = [
        "cookie", "privacy", "terms of use", "advertising", "preferences", "login", "sign up", "menu", "newsletter"
    ]
    cleaned_lines = []
    for line in text.splitlines():
        if any(junk.lower() in line.lower() for junk in junk_phrases):
            continue
        cleaned_lines.append(line)
    return " ".join(cleaned_lines)




In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time

def extract_readable_text1(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    )

    driver = webdriver.Chrome(options=chrome_options)

    # Make webdriver harder to detect
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
            Object.defineProperty(navigator, 'webdriver', {get: () => undefined});
        """
    })

    driver.get(url)
    time.sleep(3)  # Let the page start rendering

    # Scroll slowly to trigger lazy loading elements
    scroll_pause_time = 1.0
    last_height = driver.execute_script("return document.body.scrollHeight")

    for _ in range(3):  # Scroll down 3 times
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(2)  # Extra wait to be sure content is loaded

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, "html.parser")

    texts = soup.find_all(string=True)

    output = ''
    blacklist = ['[document]', 'noscript', 'header', 'html', 'meta', 'head', 'input', 'script', 'style', 'iframe', 'svg']

    for t in texts:
        if t.parent.name not in blacklist and t.strip():
            output += '{} '.format(t.strip())

    # ➡️ Now basic cleaning to remove cookie, privacy, and ads junk
    noise_keywords = [
        "Manage Preferences", "Accept All", "Cookie Settings", "Privacy Policy",
        "Personalised ads", "Terms of Use", "Ad Choices", "Consent Preferences",
        "About Our Ads", "Your Privacy", "Allow All", "Reject All", "Learn More"
    ]
    for keyword in noise_keywords:
        output = output.replace(keyword, '')

    return output.strip()

# Example usage:
""" if __name__ == "__main__":
    url = "https://coinmarketcap.com/currencies/bitcoin/"
    text = extract_readable_text(url)
    print(text)  # Show first 1000 characters
 """

' if __name__ == "__main__":\n    url = "https://coinmarketcap.com/currencies/bitcoin/"\n    text = extract_readable_text(url)\n    print(text)  # Show first 1000 characters\n '

In [17]:
def web_rag_query(query):
    print(f"🔍 Searching with Serper for: {query}")
    number_of_urls= estimate_number_of_urls(query)
    urls = web_search_serper(query,number_of_urls)

    # --- Blacklist bad URLs ---
    blacklist = [
        "https://bitflyer.com/en-us/bitcoin-chart"
    ]

    # Remove blacklisted URLs
    urls = [url for url in urls if url not in blacklist]

    print(f"📎 URLs after blacklist filter: {urls}")
    if not urls:
        return "[❌ No articles found for this query.]", urls

    print(f"📄 Extracting and summarizing content from {len(urls)} URLs...")
    summaries = []
    successful_urls = []

    for url in urls:
        try:
            raw_text = extract_readable_text(url)

            # 🧠 Now create a query-focused summarization prompt
            summarize_prompt = f"""
You are an expert crypto information extractor.

The user asked:
"{query}"

Here is a raw extracted page text:
{raw_text}

Your task is:
- Search carefully inside the page text for information **that directly helps answer the user's question**.
- Focus **only** on facts, numbers, key data points, events, and direct explanations.
- Completely **IGNORE**:
  - Menu items
  - Ads
  - Cookie policies
  - UI instructions
  - SEO keywords
  - Legal disclaimers
  - Generic unrelated text

✅ If you find useful facts:
- Extract only the relevant facts.
- Organize them as short, clean bullet points.
- Do not rephrase, do not invent — only extract.

❌ If there is no helpful information for the user's question:
- Say exactly: "No useful information found."

---
**Important:**
- Be strict: **NO summarization of the whole page**. Only answer parts matching the user's query.
- Only output the final facts, no introductions or explanations.
---
Begin extraction below:
"""




            summary = ask_llama3_locally(summarize_prompt)

            source_text = f"Source: {url}\n---\n{summary.strip()}"
            summaries.append(source_text)
            print(f"✅ Successfully summarized content from {url}")
            print("the summary is: "+summary)
            successful_urls.append(url)

        except Exception as e:
            print(f"❌ Failed to extract or summarize {url}: {e}")
            continue

    if not summaries:
        return "[❌ No accessible or relevant sources found.]", urls

    joined_summaries = "\n\n".join(summaries)

    answer_style = classify_question_type(query)

    if answer_style == "short":
        final_prompt = f"""You are a precise assistant. Give a short, factual answer based **only** on the provided sources.

### Question:
{query}

### Sources:
{joined_summaries}

### Answer:
"""
    else:
        final_prompt = f"""You are an expert crypto analyst and news summarizer.

Please answer the following question using only the information from the provided sources.
Be clear, concise, and neutral in tone.

### Question:
{query}

### Sources:
{joined_summaries}

### Summary:
"""

    print("🧠 Asking llama3 locally...")
    answer = ask_llama3_locally(final_prompt)
    return answer, successful_urls

In [25]:
query= "who are the founders and team members of Ethereum ?"
final_answer, urls =  web_rag_query(query)

display_final_answer(final_answer, query, urls)


🔍 Searching with Serper for: who are the founders and team members of Ethereum ?
🤖 AI complexity classification for 'who are the founders and team members of Ethereum ?': simple
🔍 Evaluating source: https://coinmarketcap.com/academy/article/who-founded-ethereum
🤖 AI verdict for https://coinmarketcap.com/academy/article/who-founded-ethereum: yes
🔍 Evaluating source: https://decrypt.co/36641/who-are-ethereums-co-founders-and-where-are-they-now
🤖 AI verdict for https://decrypt.co/36641/who-are-ethereums-co-founders-and-where-are-they-now: yes
🔍 Evaluating source: https://markets.businessinsider.com/news/stocks/ethereum-eth-cofounders-vitalik-buterin-cardano-charles-hoskinson-gavin-wood-2021-9
🤖 AI verdict for https://markets.businessinsider.com/news/stocks/ethereum-eth-cofounders-vitalik-buterin-cardano-charles-hoskinson-gavin-wood-2021-9: yes
🔍 Evaluating source: https://www.gemini.com/cryptopedia/co-founder-of-ethereum-founder-vitalik-buterin-ether-eth
🤖 AI verdict for https://www.gemin